In [1]:
pip install scapy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from scapy.all import *
from scapy.layers.http import HTTP, HTTPRequest

# Создание HTTP GET запроса
target_ip = "google-gruyere.appspot.com" # Или IP локального инстанса
request = IP(dst=target_ip)/TCP(dport=80, flags="A")/HTTP()/HTTPRequest(
Method='GET',
Path='/',
Host=target_ip
)

Перехват трафика

In [ ]:
    from scapy.all import sniff
    from scapy.layers.http import HTTPRequest, HTTPResponse

    def process_packet(packet):
        if packet.haslayer(HTTPRequest):
            print(f"Запрос: {packet[HTTPRequest].Host}{packet[HTTPRequest].Path}")

    sniff(filter="tcp port 80", prn=process_packet, store=0)


Запрос: b'www.msftconnecttest.com'b'/connecttest.txt'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'149.154.167.222:80'b'/api'
Запрос: b'149.154.167.222:80'b'/api'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'www.msftconnecttest.com'b'/connecttest.txt'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'www.msftconnecttest.com'b'/connecttest.txt'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'www.msftconnecttest.com'b'/connecttest.txt'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'149.154.166.111:80'b'/api'
Запрос: b'149.154.166.111:80'b'/api'
Запрос: b'ocsp.sectigo.com'b'/MFEwTzBNMEswSTAJBgUrDgMCGgUABBQzZq5DY2RqjBUGCjtK3%2FEujBgJrwQU0SLaTFnxS18mOKqd1u7rDcP7qWECEDbgWe2IjdV67dVwtnJ%2Fm9o%3D'
Запрос: b'www.msftconnecttest.com'b'/connecttest.txt'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'91.105.192.100:80'b'/api'
Запрос: b'149.154.167.222:80'b'/api'

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [ ]:
def detect_xss_in_response(pkt):
    if pkt.haslayer(Raw) and pkt.haslayer(IP):
        payload = pkt[Raw].load.decode(errors='ignore')
        
        # Ищем признаки атаки в трафике
        detectors = ["<script>alert", "onerror=alert", "javascript:alert"]
        
        for d in detectors:
            if d in payload:
                # Если пакет идет ОТ сервера к НАМ (порт 80 -> наш порт)
                if pkt[TCP].sport == 80:
                    print(f"\n[!!!] XSS СРАБОТАЛА В ОТВЕТЕ СЕРВЕРА!")
                    print(f"Сервер {pkt[IP].src} прислал исполняемый код:")
                    print("-" * 40)
                    print(payload)
                    print("-" * 40)

sniff(filter="tcp port 80", prn=detect_xss_in_response, store=0)